In [54]:
import h5py
from deeprankcore.DataSet import save_hdf5_keys
import torch
from models import CnnClassificationBaseline
from deeprank.learn import DataSet, NeuralNet
import torch.utils.data as data_utils
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

In [2]:
hdf5_path = '000_hla_drb1_0101_15mers.hdf5'
sample_path = 'one_sample.hdf5'
pretrained_model = 'best_valid_model.pth.tar'

In [6]:
save_hdf5_keys(hdf5_path, ['BA_105966'], sample_path, hardcopy = True)

In [3]:
model = NeuralNet(sample_path, CnnClassificationBaseline, pretrained_model = pretrained_model, outdir='./out/')
model.test()



=	 DeepRank Data Set
=
=	 Training data
=	 -> one_sample.hdf5
=
=
=

   Checking dataset Integrity


   Processing data set:
   Train dataset
      Computing norm for one_sample.hdf5
 : STD is null for PSSM_ARG_chain2 in one_sample.hdf5
 : STD is null for PSSM_CYS_chain2 in one_sample.hdf5
 : STD is null for PSSM_GLN_chain2 in one_sample.hdf5
 : STD is null for PSSM_GLU_chain2 in one_sample.hdf5
 : STD is null for PSSM_HIS_chain2 in one_sample.hdf5
 : STD is null for PSSM_ILE_chain2 in one_sample.hdf5
 : STD is null for PSSM_MET_chain2 in one_sample.hdf5
 : STD is null for PSSM_PHE_chain2 in one_sample.hdf5
 : STD is null for PSSM_PRO_chain2 in one_sample.hdf5
 : STD is null for PSSM_TRP_chain2 in one_sample.hdf5
  Final STD Null for Feature_ind/PSSM_ARG_chain2. Changed it to 1
  Final STD Null for Feature_ind/PSSM_CYS_chain2. Changed it to 1
  Final STD Null for Feature_ind/PSSM_GLN_chain2. Changed it to 1
  Final STD Null for Feature_ind/PSSM_GLU_chain2. Changed it to 1
  Final STD

In [5]:
model.data_set.input_shape

(64, 35, 30, 30)

In [ ]:
# step by step without using NeuralNet

In [15]:
data_set = DataSet(
    'one_sample.hdf5',
    chain1="M",
    chain2="P",
    process=False)

state = torch.load(pretrained_model,  map_location='cpu')

data_set.select_feature = state['select_feature']
data_set.select_target = state['select_target']

data_set.pair_chain_feature = state['pair_chain_feature']
data_set.dict_filter = state['dict_filter']

data_set.normalize_targets = state['normalize_targets']
if data_set.normalize_targets:
    data_set.target_min = state['target_min']
    data_set.target_max = state['target_max']

data_set.normalize_features = state['normalize_features']
if data_set.normalize_features:
    data_set.feature_mean = state['feature_mean']
    data_set.feature_std = state['feature_std']

data_set.transform = state['transform']
data_set.proj2D = state['proj2D']
data_set.target_ordering = state['target_ordering']
data_set.clip_features = state['clip_features']
data_set.clip_factor = state['clip_factor']
data_set.mapfly = state['mapfly']
data_set.grid_info = state['grid_info']

data_set.process_dataset()



=	 DeepRank Data Set
=
=	 Training data
=	 -> one_sample.hdf5
=
=
=

   Checking dataset Integrity


   Processing data set:
   Train dataset
 : STD is null for PSSM_ARG_chain2 in one_sample.hdf5
 : STD is null for PSSM_CYS_chain2 in one_sample.hdf5
 : STD is null for PSSM_GLN_chain2 in one_sample.hdf5
 : STD is null for PSSM_GLU_chain2 in one_sample.hdf5
 : STD is null for PSSM_HIS_chain2 in one_sample.hdf5
 : STD is null for PSSM_ILE_chain2 in one_sample.hdf5
 : STD is null for PSSM_MET_chain2 in one_sample.hdf5
 : STD is null for PSSM_PHE_chain2 in one_sample.hdf5
 : STD is null for PSSM_PRO_chain2 in one_sample.hdf5
 : STD is null for PSSM_TRP_chain2 in one_sample.hdf5
  Final STD Null for Feature_ind/PSSM_ARG_chain2. Changed it to 1
  Final STD Null for Feature_ind/PSSM_CYS_chain2. Changed it to 1
  Final STD Null for Feature_ind/PSSM_GLN_chain2. Changed it to 1
  Final STD Null for Feature_ind/PSSM_GLU_chain2. Changed it to 1
  Final STD Null for Feature_ind/PSSM_HIS_chain2. Ch

In [16]:
data_set.input_shape

(64, 35, 30, 30)

In [45]:
net = CnnClassificationBaseline(data_set.input_shape)
device = torch.device("cpu")

net.to(device)

if state['cuda']:
    for paramname in list(state['state_dict'].keys()):
        paramname_new = paramname.lstrip('module.')
        if paramname != paramname_new:
            state['state_dict'][paramname_new] = \
                state['state_dict'][paramname]
            del state['state_dict'][paramname]

net.load_state_dict(state['state_dict'])

<All keys matched successfully>

In [47]:
optimizer = optim.SGD(
    net.parameters(),
    lr=0.005,
    momentum=0.9,
    weight_decay=0.001)
optimizer.load_state_dict(state['optimizer'])

In [48]:
index = list(range(data_set.__len__()))
sampler = data_utils.sampler.SubsetRandomSampler(index)
loader = data_utils.DataLoader(data_set, sampler=sampler)

In [49]:
for idx, data in enumerate(loader):
    print(data['feature'].shape)

torch.Size([1, 64, 35, 30, 30])


In [51]:
net.train(mode=False)
torch.set_grad_enabled(False)

In [59]:
for d in loader:
    inputs = d['feature']
    targets = d['target']
    mol = d['mol']
    inputs, targets = Variable(inputs).float(), Variable(targets).float()
    targets = targets.long()
    print(mol)
    print(inputs.shape)
    print(targets)
    outputs = net(inputs)
    print(outputs)
    targets = targets.view(-1)
    print(targets)
    # F.softmax(torch.FloatTensor(out), dim=1).data.numpy()[:, 1]

[('one_sample.hdf5',), ('BA_105966',)]
torch.Size([1, 64, 35, 30, 30])
tensor([[0]])
tensor([[ 1.9637, -1.9273]])
tensor([0])


In [60]:
inputs_toplot = np.squeeze(np.array(inputs))
inputs_toplot.shape


(64, 35, 30, 30)

In [63]:
one_vol = inputs_toplot[0, :, :, :]
one_vol.shape

(35, 30, 30)